In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import keras
import numpy as np
import pandas as pd
params = ['Px', 'Py', 'Pz', 'E', 'D0', 'DZ', 'X', 'Y', 'Z']
max_len = 100

Using TensorFlow backend.


In [2]:
def pad_values(vals, val = 0, max_len = max_len):
    sr, sc = vals.shape
    if max_len - sr > 0:
        return np.pad(vals, ((0, max_len - sr), (0, 0)), mode='constant')
    return vals[:, :max_len, :]

def print_accuracy( p, target ):
    p_cat = np.argmax(p,axis=1)
    test_target = np.argmax(target, axis = 1)
    print "Fraction of good prediction"
    print len(np.where( p_cat == test_target)[0])
    print len(np.where( p_cat == test_target )[0])/float(len(p_cat)),"%"

def accuracy(p, target):
    p_cat = np.argmax(p,axis=1)
    test_target = np.argmax(target, axis = 1)
    return len(np.where( p_cat == test_target)[0])/float(len(p_cat))
    
def df_to_target(df, output, params = params):
    print "lookup"
    training = [df.loc[df['njet']==j][params].sort_values(['D0', 'DZ'], ascending = False) 
                    for j in df.njet.unique()]
    print "to numpy"
    training = np.array([pad_values(i.values) for i in training])
    training_target = np.array([output for i in range(len(training))])
    return training, training_target
    
def df_njet_index(df):
    """Combine event # and jet #"""
    events_jets = df[['event', 'jet']].values
    njet = np.zeros(events_jets.shape[0])
    prev = np.array([-1, -1])
    count = -1
    for ind, val in enumerate(events_jets):
        if any(prev != val):
            count += 1
            prev = val
        njet[ind] = count
    df['njet'] = njet
    
def h5_to_target(fname, output, params = params):
    df = pd.read_hdf(fname)
    df_njet_index(df)
    return df_to_target(df, output, params)

def make_test_split(training, target, test_size = 200):
    """Split training/target into training/target and test/target"""
    num = training.shape[0]
    indices = np.random.choice(range(num), test_size, replace = False)
    test = training[indices]
    test_target = target[indices]
    training = np.delete(training, indices, axis = 0)
    target = np.delete(target, indices, axis = 0)
    return training, target, test, test_target

def get_training_target_sample(training, target, sample_size):
    indices = np.random.choice(range(training.shape[0]), sample_size, replace = False)
    ntraining = training[indices]
    ntarget = target[indices]
    return ntraining, ntarget

def shuffle_together(training, target):
    p = np.random.permutation(len(training))
    return training[p], target[p]

def combine_sets(sets, sample_size = 10000):
    """Combine a list of (training, target, test, test_target) sets"""
    a = min([i[0].shape[0] for i in sets])
    sample_size = min(a, sample_size)
    samples = [get_training_target_sample(i[0], i[1], sample_size) for i in sets]
    ntraining = np.concatenate([i[0] for i in samples])
    ntarget = np.concatenate([i[1] for i in samples])
    ntest = np.concatenate([i[2] for i in sets])
    ntest_target = np.concatenate([i[3] for i in sets])
    ntraining, ntarget = shuffle_together(ntraining, ntarget)
    ntest, ntest_target = shuffle_together(ntest, ntest_target)
    return ntraining, ntarget, ntest, ntest_target

In [3]:
a = h5_to_target('qcd_out.h5', output = [1, 0])
b = h5_to_target('higgs_out.h5', output = [0, 1])
qcd = make_test_split(*a)
higgs = make_test_split(*b)
training, training_target, test, test_target = combine_sets([qcd, higgs], sample_size = 5000)

lookup
to numpy
lookup
to numpy


In [7]:
from keras.models import Sequential
from keras.layers import Masking, Dense, Dropout, Activation, Reshape
from keras.layers.recurrent import LSTM
m = Sequential()
m.add(Masking(0, input_shape = (max_len, len(params))))
m.add(LSTM(100))
m.add(Dense(2))
m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
for i in range(1000, 1001, 1000):
    print "Training using %s training points" % i
    s = time.time()
    m.fit(training[:i], training_target[:i], nb_epoch=10, verbose = 0)
    e = time.time()
    print "Total time: %s" % (e - s)
    print_accuracy(m.predict(test), test_target)

Training using 1000 training points
Total time: 28.8684048653
Fraction of good prediction
235
0.5875 %


In [ ]:
m.predict(test[:1])

In [ ]:
a = m.predict(test)
for ind, t in enumerate(test_target):
    print a[ind], t

In [ ]:
accuracy(m.predict(test), test_target)

In [5]:
import time
time.time()

1500919110.882467

In [6]:
keras.backend.backend()

'tensorflow'

Theano training time: 188.917440891 

Tensorflow (CPU) training time: 28.8684048653